In [ ]:
#default_exp config
%load_ext autoreload
from nbdev.showdoc import show_doc
from utilities.ipynb_docgen import *
from wtlike.config import *

# Configuration data and basic functions
> Basic functions and configuration stuff

In [ ]:
#collapse-hide
def summary():
    """
    
    ##### config.Config -- parameters

    for now, but could be obtained from an editable yaml file.
    
    {config_text}
    
    ##### config.Files -- default file paths
    {files_text}
 
    These are dataclass classes, and can be initialized with
    parameters. For example, to suppress printout 
        
    ```
    config = Config(verbose=0)
    ```
    
    ##### config.cache -- a file cache
    The class `Cache`, available from `config.cache` implements a file cache in the folder
    `config.files.cachepath`
    
    {cache_text}
    """
    config = Config()
    config_text = monospace(config, summary='config parameter list')
    files_text = monospace(Files(), summary='file list')
    files = Files()
 
    cache_text = monospace(config.cache, 'cache contents' )
    assert files.valid
    return locals()
if Files().valid:
    nbdoc(summary) 

##### config.Config -- parameters

for now, but could be obtained from an editable yaml file.

<details  class="nbdoc-description" >  <summary> config parameter list </summary>  <div style="margin-left: 5%"><pre>Configuration parameters <br>  verbose         : 3<br>  cachepath       : /tmp/cache<br>  mjd_range       : None<br>  radius          : 7<br>  cos_theta_max   : 0.4<br>  z_max           : 100<br>  time_interval   : 1<br>  use_uint8       : False<br>  nside           : 1024<br>  nest            : True<br>  bins_per_decade : 4<br>  base_spectrum   : lambda E: (E/1000)**-2.1<br>  energy_range    : (100.0, 1000000.0)<br>  likelihood_rep  : poisson<br></pre></div> </details>

##### config.Files -- default file paths
<details  class="nbdoc-description" >  <summary> file list </summary>  <div style="margin-left: 5%"><pre>File paths for light curves<br>  data       : /home/burnett/data<br>  ft2        : /home/burnett/work/lat-data/ft2<br>  gti        : /home/burnett/work/lat-data/binned<br>  aeff       : /home/burnett/work/lat-data/aeff<br>  weights    : /home/burnett/onedrive/fermi/weight_files<br></pre></div> </details>

These are dataclass classes, and can be initialized with
parameters. For example, to suppress printout 
    
```
config = Config(verbose=0)
```

##### config.cache -- a file cache
The class `Cache`, available from `config.cache` implements a file cache in the folder
`config.files.cachepath`

<details  class="nbdoc-description" >  <summary> cache contents </summary>  <div style="margin-left: 5%"><pre>Cache contents<br> key                          size  time                 name, in folder /tmp/cache<br>  gti                      1018321  2020-12-29 05:55     cache_file_3fa07cccddc0cbc6.pkl<br>  photons_Geminga         22334715  2020-12-29 05:57     cache_file_55c429e70273e19.pkl<br>  exposure_Geminga        86263782  2020-12-29 06:00     cache_file_6b9b5aabba763673.pkl<br>  binexp_Geminga             64450  2020-12-29 06:00     cache_file_16e94afb7a0c640.pkl<br>  cells_Geminga            4746232  2020-12-29 06:00     cache_file_99633b83cb9c28b.pkl<br>  lightfcurve_Geminga      6887303  2020-12-29 06:00     cache_file_13e6086314117a1.pkl<br>  photons_3C 279           3659059  2020-12-29 06:02     cache_file_f5649e6474dea2a.pkl<br>  exposure_3C 279         78040998  2020-12-29 06:04     cache_file_11471b8a6311d7d.pkl<br>  binexp_3C 279              64434  2020-12-29 06:04     cache_file_2c0392150e5c66.pkl<br>  cells_3C 279             1001088  2020-12-29 06:04     cache_file_37c8297c61b76a4.pkl<br>  lightfcurve_3C 279       4008017  2020-12-29 06:04     cache_file_4268cd9a8a907e4a.pkl<br>  weight_hist_Geminga          558  2021-01-17 15:53     cache_file_0965548252135b4.pkl<br></pre></div> </details>


In [ ]:
#export
from astropy.time import Time
from astropy.coordinates import SkyCoord
from dataclasses import dataclass
from pathlib import Path
from typing import Tuple
import os, sys
import numpy as np
import pickle

In [ ]:
#export
class Cache(dict):
    """
    Manage a file cache

    - `path` -- string or `filepath` object <br> This is the folder where the index and data files are saved.
    - `clear` -- set True to clear the cache on initialization
    
    This uses pickle to save objects, associated with a hashable key, which is used to index the
    filename in a file `index.pkl` in the same folder. 
    
    The `__call__` function is a convenient way to use it, so one call may either store a new entry or retrieve an existing one.

    """
    
    def __init__(self, path, clear:bool=False):


        self.path = Path(path) if path else None
        if not self.path: return
        self.path.mkdir(exist_ok=True)
        assert self.path.exists()
        self.index_file = self.path/'index.pkl'

        if self.path.exists():  
            if clear: 
                print('Clearing cache!')
                self.clear()
            else:
                self._load_index()
        else:
            self.path.mkdir(exist_ok=True)        

    def _dump_index(self):
        with open(self.index_file, 'wb') as file:
            pickle.dump(self, file)
        
    def _load_index(self):
        if not self.index_file.exists(): 
            self._dump_index()
            return
        with open(self.index_file, 'rb') as file:
            self.update(pickle.load(file))
        
    def add(self, key, object, exist_ok=False):
        if not self.path: return
        if key  in self:
            if not exist_ok:
                print(f'Warning: cached object for key "{key}" exists', file=sys.stderr)
            filename = self[key]
        else:
            filename = self.path/f'cache_file_{hex(key.__hash__())[3:]}.pkl'
            self[key] = filename
            self._dump_index() 
 
        with open(filename, 'wb') as file:
            pickle.dump(object, file )

        
    def get(self, key):
        if key not in self:
            return None
        filename = self[key]
        if not filename.exists():
            # perhaps deleted by another instance?
            print(f'File for Cache key {key} not found, removing entry', file='sys.stderr')
            selt.pop(key)
            return None
        with open(filename, 'rb') as file:
            ret = pickle.load(file)
        return ret
    
    def clear(self):
        if not self.path: return
        for f in self.path.iterdir():
            if f.is_file: 
                f.unlink()
        super().clear()

        self._dump_index()
        
    def remove(self, key):
        """remove entry and associated file"""
        if not self.path: return
        if key not in self:
            print(f'Cache: key {key} not found', file=sys.stderr)
            return
        filename = self[key]
        filename.unlink()
        super().pop(key)
        self._dump_index()
        
                
    def __call__(self, key, func, *pars, description='', overwrite=False, **kwargs,
                ):
        """
        One-line usage interface for cache use
        
        - `key` -- key to use, usually a string. Must be hashable <br>
            If none, ignore cache and return the function evaluation
        - `func` -- user function that will return an object that can be pickled
        - `pars`, `kwargs` -- pass to `func`
        - `description` -- optional string that will be printed

        Example:
        <pre>
        mycache = Cache('/tmp/thecache', clear=True)
        
        def myfun(x):
            return x

        result = mycache('mykey', myfun, x=99,  description='My data')

        </pre>

        """
        
        if key is None:
            return func(*pars, **kwargs)
        
        ret = self.get(key)
        if description:
            print(f'{description}: {"Saving to" if key not in self else "Restoring from"} cache with key "{key}"')

        if ret is None or overwrite: 
            ret = func(*pars, **kwargs)
            self.add(key, ret)
        return ret
             
    def __str__(self):
        import datetime
        if not self.path: return 'Cache not enabled'
        s = f'Cache contents\n {"key":20}   {"size":>10}  {"time":20} {"name"}, in folder {self.path}\n'
        for name, value in self.items():
            if name is None: continue
            stat = value.stat()
            size = stat.st_size
            mtime= str(datetime.datetime.fromtimestamp(stat.st_mtime))[:16]
            s += f'  {name:20s}  {size:10}  {mtime:20} {value.name}\n'
        return s

In [ ]:
show_doc(Cache.__call__)

<h4 id="Cache.__call__" class="doc_header"><code>Cache.__call__</code><a href="__main__.py#L94" class="source_link" style="float:right">[source]</a></h4>

> <code>Cache.__call__</code>(**`key`**, **`func`**, **\*`pars`**, **`description`**=*`''`*, **`overwrite`**=*`False`*, **\*\*`kwargs`**)

One-line usage interface for cache use

- `key` -- key to use, usually a string. Must be hashable <br>
    If none, ignore cache and return the function evaluation
- `func` -- user function that will return an object that can be pickled
- `pars`, `kwargs` -- pass to `func`
- `description` -- optional string that will be printed

Example:
<pre>
mycache = Cache('/tmp/thecache', clear=True)

def myfun(x):
    return x

result = mycache('mykey', myfun, x=99,  description='My data')

</pre>

### Cache test

In [ ]:
#collapse_hide
def cache_test(path):
    c = Cache(path, clear=True)

    # simmple interface
    c.add('one', 'one');
    c.add('two', 'two')
    c.add('two', 'two') # getnerates warning
    if path is not None:
        assert c.get('two') == 'two'

    # test function interface
    func = lambda x:f'value: {x}'
    
    r1 = c('four',  func,  4, description='Test')
    r2 = c('four',  func,  5,  description='Test') #should not get called
    assert c.path is None or r1==r2, f'{r1}, {r2}'
    
    # remaving an entry
    print(f'Before remove:\n{c}')
    assert 'four' in c
    c.remove('four')
    assert 'four' not in c
   
    
    c.clear()
cache_test('/tmp/test_cache')
# disabled should ignore
#cache_test(None)

Clearing cache!
Test: Saving to cache with key "four"
Test: Restoring from cache with key "four"
Before remove:
Cache contents
 key                          size  time                 name, in folder /tmp/test_cache
  one                           13  2021-01-18 09:10     cache_file_3ad148db70fdd696.pkl
  two                           13  2021-01-18 09:10     cache_file_6eedb81db9bf4df.pkl
  four                          18  2021-01-18 09:10     cache_file_25e7e89ed95f21f.pkl



## Configuration data classes


In [ ]:
#export
@dataclass
class Files:
    """ paths to the various files that we need"""

    data:str = '$HOME/data'
    ft2: str = '$HOME/work/lat-data/ft2'
    gti: str = '$HOME/work/lat-data/binned/'
    aeff:str = '$HOME/work/lat-data/aeff'
    weights: str = '$HOME/onedrive/fermi/weight_files'

    # expand -- not implemented in Path
    def __post_init__(self):
        d = self.__dict__
        for name, value in d.items():
            d[name] = Path(os.path.expandvars(value))
        
    @property
    def valid(self):
        """assume all files ok if aeff"""
        return self.aeff.exists()
    
    def __repr__(self):
        s = 'File paths for light curves\n'
        for name, value in self.__dict__.items():
            s += f'  {name:10s} : {value}\n'
        return s

In [ ]:
Files()

File paths for light curves
  data       : /home/burnett/data
  ft2        : /home/burnett/work/lat-data/ft2
  gti        : /home/burnett/work/lat-data/binned
  aeff       : /home/burnett/work/lat-data/aeff
  weights    : /home/burnett/onedrive/fermi/weight_files

In [ ]:
#export
@dataclass
class Config:
    """Default light curve configuration parameters"""
    verbose : int = 3
    files :'' =  None

    # cache
    cachepath: str = '/tmp/cache'
        
    # photon selection
    mjd_range : Tuple=None
    radius: float = 7
    cos_theta_max:float=0.4
    z_max : float=100

    # binning
    energy_edges = np.logspace(2,6,17)
    etypes = (0,1)
    time_interval: int = 1
    use_uint8: bool=False  # for weights 

    # healpix data representation used by data
    nside : int=1024
    nest: bool=True

    # exposure calculation
    bins_per_decade: int=4
    base_spectrum: str='lambda E: (E/1000)**-2.1'
    energy_range: Tuple = (100.,1e6)
        
    # analysis
    likelihood_rep: str='poisson'
        
    def __post_init__(self):
        if self.files is None: self.files=Files()

    @property
    def cache(self):
        if not hasattr(self, '_cache'):
            self._cache = Cache(self.cachepath, clear=False)
        return self._cache
        
    @property
    def valid(self):
        return self.files.valid 
    
    def __str__(self):
        s = 'Configuration parameters \n'
        for name, value in self.__dict__.items():
            if name=='files' or name.startswith('_'): continue
            s += f'  {name:15s} : {value}\n'
        return s

    def __repr__(self): return str(self)

In [ ]:
config = Config()
print (config, config.files, config.cache)

Configuration parameters 
  verbose         : 3
  cachepath       : /tmp/cache
  mjd_range       : None
  radius          : 7
  cos_theta_max   : 0.4
  z_max           : 100
  time_interval   : 1
  use_uint8       : False
  nside           : 1024
  nest            : True
  bins_per_decade : 4
  base_spectrum   : lambda E: (E/1000)**-2.1
  energy_range    : (100.0, 1000000.0)
  likelihood_rep  : poisson
 File paths for light curves
  data       : /home/burnett/data
  ft2        : /home/burnett/work/lat-data/ft2
  gti        : /home/burnett/work/lat-data/binned
  aeff       : /home/burnett/work/lat-data/aeff
  weights    : /home/burnett/onedrive/fermi/weight_files
 Cache contents
 key                          size  time                 name, in folder /tmp/cache
  gti                      1018321  2020-12-29 05:55     cache_file_3fa07cccddc0cbc6.pkl
  photons_Geminga         22334715  2020-12-29 05:57     cache_file_55c429e70273e19.pkl
  exposure_Geminga        86263782  2020-12-29 06:00

## Time conversion

- MET: mission elapsed time
- MJD: modified Julian date (days)
- UTC: ISO time

In [ ]:
#export

day = 24*3600.
first_data=54683

def MJD(met):
    "convert MET to MJD"
    #mission_start = Time('2001-01-01T00:00:00', scale='utc').mjd
    # From a FT2 file header
    # MJDREFI =               51910. / Integer part of MJD corresponding to SC clock S
    # MJDREFF =  0.00074287037037037 / Fractional part of MJD corresponding to SC cloc
    mission_start = 51910.00074287037
    return (mission_start + met/day  )

def UTC(mjd):
    " convert MJD value to ISO date string"
    t=Time(mjd, format='mjd')
    t.format='iso'; t.out_subfmt='date_hm'
    return t.value

In [ ]:
assert UTC(MJD(0))=='2001-01-01 00:01'

In [ ]:
#export                
class PointSource():
    """Manage the position and name of a point source
    """
    def __init__(self, name, position=None):
        """position: (l,b) tuple or None. if None, expect to be found by lookup
        """
        self.name=name
        if position is None:
            skycoord = SkyCoord.from_name(name)
            gal = skycoord.galactic
            self.l,self.b = (gal.l.value, gal.b.value)
        else:
            self.l,self.b = position
            skycoord = SkyCoord(self.l,self.b, unit='deg', frame='galactic')
        self.skycoord = skycoord
    def __str__(self):
        return f'Source "{self.name}" at: (l,b)=({self.l:.3f},{self.b:.3f})'
    def __repr__(self): return str(self)

    @property
    def ra(self):
        sk = self.skycoord.transform_to('fk5')
        return sk.ra.value
    @property
    def dec(self):
        sk = self.skycoord.transform_to('fk5')
        return sk.dec.value

    @property
    def filename(self):
        """Modified name for file system"""
        return self.name.replace(' ', '_').replace('+','p')

    @classmethod
    def fk5(cls, name, position):
        """position: (ra,dec) tuple """
        ra,dec = position
        sk = SkyCoord(ra, dec, unit='deg',  frame='fk5').transform_to('galactic')
        return cls(name, (sk.l.value, sk.b.value))

In [ ]:
show_doc(PointSource.fk5)

<h4 id="PointSource.fk5" class="doc_header"><code>PointSource.fk5</code><a href="__main__.py#L35" class="source_link" style="float:right">[source]</a></h4>

> <code>PointSource.fk5</code>(**`name`**, **`position`**)

position: (ra,dec) tuple 

In [ ]:
for s, expect in [( PointSource('Geminga'),             'Source "Geminga" at: (l,b)=(195.134,4.266)'),
                  ( PointSource('gal_source', (0,0)),   'Source "gal_source" at: (l,b)=(0.000,0.000)', ),
                  ( PointSource.fk5('fk5_source',(0,0)),'Source "fk5_source" at: (l,b)=(96.337,-60.189)',)
                   ]:    
    assert str(s)==expect, f'expected {expect}, got {str(s)}'


In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()
!date

Converted 00_config.ipynb.
Converted 01-data_man.ipynb.
Converted 01_effective_area.ipynb.
Converted 02_gti.ipynb.
Converted 03_exposure.ipynb.
Converted 04_photon_data.ipynb.
Converted 05_weights.ipynb.
Converted 06_poisson.ipynb.
Converted 07_cells.ipynb.
Converted 08_loglike.ipynb.
Converted 09_lightcurve.ipynb.
Converted 10_simulation.ipynb.
Converted 14_bayesian.ipynb.
Converted index.ipynb.
Mon Jan 18 09:11:02 PST 2021
